<a href="https://colab.research.google.com/github/Basha15/Amdox-Internship/blob/main/Crypto_Analysis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("⏳ Installing necessary libraries for the environment... (This may take a minute)")
try:
    import yfinance
    import prophet
    import plotly
except ImportError:
    install("yfinance")
    install("prophet")
    install("plotly")
    install("pandas")
    print("✅ Libraries installed successfully!")

# --- STEP 2: IMPORTS ---
import pandas as pd
import numpy as np
import yfinance as yf
from prophet import Prophet
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import warnings

warnings.filterwarnings('ignore')

# --- STEP 3: DATA COLLECTION MODULE ---
def fetch_crypto_data(ticker='BTC-USD', period='2y'):
    """
    Fetches historical cryptocurrency data using Yahoo Finance API.
    Refers to PDF Section 1: Cryptocurrency Data Collection.
    """
    print(f"\n📡 Fetching data for {ticker}...")
    data = yf.download(ticker, period=period, interval='1d', progress=False)

    if data.empty:
        raise ValueError("No data found. Check your internet connection or ticker symbol.")

    data.reset_index(inplace=True)

    # Handle MultiIndex columns if present (newer yfinance versions)
    if isinstance(data.columns, pd.MultiIndex):
         data.columns = data.columns.get_level_values(0)

    # Ensure Date column is datetime
    data['Date'] = pd.to_datetime(data['Date'])

    # Calculate Daily Returns & Volatility
    data['Returns'] = data['Close'].pct_change()
    data['Volatility_30d'] = data['Returns'].rolling(window=30).std() * np.sqrt(365) # Annualized Volatility

    # Simple Moving Averages for Analysis
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()

    print(f"✅ Data fetched: {len(data)} rows loaded.")
    return data

# --- STEP 4: FORECASTING MODEL (PROPHET) ---
def generate_forecast(data, days_ahead=90):
    """
    Trains a Time Series model to predict future prices.
    Refers to PDF Section 3: Time Series Forecasting Models.
    """
    print(f"🧠 Training Prophet Model for {days_ahead} days forecast...")

    # Prepare data for Prophet (requires columns 'ds' and 'y')
    df_train = data[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})

    # Initialize and fit model
    model = Prophet(daily_seasonality=True, yearly_seasonality=True)
    model.fit(df_train)

    # Create future dataframe
    future = model.make_future_dataframe(periods=days_ahead)
    forecast = model.predict(future)

    print("✅ Forecasting complete.")
    return forecast, model

# --- STEP 5: VISUALIZATION GUI (PLOTLY) ---
def create_dashboard(data, forecast, ticker):
    """
    Creates an interactive dashboard.
    Refers to PDF Section 4: Graphical User Interface (GUI).
    """
    print("\n🎨 Generating Interactive Dashboard...")

    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.5, 0.25, 0.25],
        subplot_titles=(f"{ticker} Price Forecast & Trends", "Market Volatility (Risk Analysis)", "Trading Volume")
    )

    # 1. Main Price Chart with Forecast
    # Historical Data
    fig.add_trace(go.Scatter(x=data['Date'], y=data['Close'], name='Actual Price',
                             line=dict(color='blue', width=1)), row=1, col=1)

    # Forecast Data
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicted Price',
                             line=dict(color='green', dash='dot')), row=1, col=1)

    # Uncertainty Intervals (Upper/Lower bounds)
    fig.add_trace(go.Scatter(
        x=forecast['ds'].tolist() + forecast['ds'].tolist()[::-1],
        y=forecast['yhat_upper'].tolist() + forecast['yhat_lower'].tolist()[::-1],
        fill='toself',
        fillcolor='rgba(0,255,0,0.1)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    ), row=1, col=1)

    # Moving Averages
    fig.add_trace(go.Scatter(x=data['Date'], y=data['SMA_50'], name='50-Day SMA',
                             line=dict(color='orange', width=1)), row=1, col=1)

    # 2. Volatility Chart
    fig.add_trace(go.Scatter(x=data['Date'], y=data['Volatility_30d'], name='Annualized Volatility',
                             line=dict(color='red', width=1), fill='tozeroy'), row=2, col=1)

    # 3. Volume Chart
    fig.add_trace(go.Bar(x=data['Date'], y=data['Volume'], name='Volume', marker_color='purple'), row=3, col=1)

    # Layout Styling
    fig.update_layout(
        title=f"<b>CryptoPulse Analytics Dashboard: {ticker}</b><br><sup>Project for Data Analytics Internship | Powered by Python, Prophet & Plotly</sup>",
        template="plotly_dark",
        height=900,
        hovermode="x unified",
        legend=dict(orientation="h", y=1.02, xanchor="right", x=1)
    )

    fig.show()

# --- STEP 6: EXECUTION ---
# This is the main function that orchestrates the project.

if __name__ == "__main__":
    # Configuration
    TICKER = "BTC-USD"  # You can change this to 'ETH-USD' or 'SOL-USD'
    PERIOD = "2y"       # 2 years of historical data

    try:
        # 1. Get Data
        df = fetch_crypto_data(TICKER, PERIOD)

        # 2. Generate Predictions
        forecast_df, model = generate_forecast(df)

        # 3. Show Dashboard
        create_dashboard(df, forecast_df, TICKER)

        # 4. Generate Summary Report for HR
        current_price = df['Close'].iloc[-1]
        predicted_price = forecast_df['yhat'].iloc[-1]
        trend = "UPWARD" if predicted_price > current_price else "DOWNWARD"

        print("\n" + "="*50)
        print("📊 EXECUTIVE SUMMARY (HR/INTERNSHIP REPORT)")
        print("="*50)
        print(f"🔹 Asset Analyzed: {TICKER}")
        print(f"🔹 Current Price: ${current_price:,.2f}")
        print(f"🔹 90-Day Forecast: ${predicted_price:,.2f}")
        print(f"🔹 Predicted Trend: {trend}")
        print(f"🔹 Volatility Risk: {df['Volatility_30d'].iloc[-1]*100:.2f}%")
        print("-" * 50)
        print("✅ Project executed successfully. This demonstrates skills in:")
        print("   - API Data Mining (yfinance)")
        print("   - Time Series Forecasting (Facebook Prophet)")
        print("   - Technical Analysis (SMA, Volatility)")
        print("   - Interactive Data Visualization (Plotly)")
        print("="*50)

    except Exception as e:
        print(f"\n❌ Error occurred: {e}")

⏳ Installing necessary libraries for the environment... (This may take a minute)

📡 Fetching data for BTC-USD...
✅ Data fetched: 732 rows loaded.
🧠 Training Prophet Model for 90 days forecast...
✅ Forecasting complete.

🎨 Generating Interactive Dashboard...



📊 EXECUTIVE SUMMARY (HR/INTERNSHIP REPORT)
🔹 Asset Analyzed: BTC-USD
🔹 Current Price: $88,962.68
🔹 90-Day Forecast: $26,054.86
🔹 Predicted Trend: DOWNWARD
🔹 Volatility Risk: 47.94%
--------------------------------------------------
✅ Project executed successfully. This demonstrates skills in:
   - API Data Mining (yfinance)
   - Time Series Forecasting (Facebook Prophet)
   - Technical Analysis (SMA, Volatility)
   - Interactive Data Visualization (Plotly)
